# 🛡️ Estrategia Viernes: Minimización de Riesgo Selectiva ("Búnker de Calidad")

## 1. Diagnóstico y Justificación Táctica
Tras la autopsia de la sesión anterior, hemos identificado la materialización de un **Riesgo Idiosincrático Extremo**. La estrategia de *Momentum* colapsó debido a una reversión violenta de la tendencia en los activos de mayor beta (Kurtosis > 19), confirmando un **Cambio de Régimen** en el mercado.

* **La Decisión:** Pivotamos de un enfoque de Maximización de Utilidad a uno de **Preservación de Capital**. El objetivo prioritario es blindar el denominador del Ratio Sharpe (Volatilidad), eliminando la exposición a "ruido" y asegurando que solo activos con esperanza matemática positiva entren en la cartera.

## 2. Metodología: Filtrado de Higiene Financiera

Para corregir el defecto estructural de los modelos clásicos de Mínima Varianza (que tienden a asignar peso a activos estables pero con rentabilidad negativa), el código ejecuta un protocolo de **Saneamiento del Universo de Inversión** antes de la optimización.

El algoritmo define un subconjunto elegible ($\mathbb{S}$) aplicando un filtro de calidad estricto sobre los 50 activos disponibles:

### FILTRO PREVIO: Esperanza Matemática Positiva
Se excluyen automáticamente todos los activos que destruyen valor estructuralmente (retornos negativos), independientemente de su estabilidad.

$$
\mathbb{S} = \{ i \in \text{Activos} \mid \mu_i > 0 \}
$$

* **Razón Táctica:** El Ratio Sharpe es una fracción ($Retorno/Riesgo$). Al filtrar matemáticamente los activos con $\mu \le 0$, aseguramos que la contribución al numerador de la ecuación sea siempre positiva, evitando el "lastre" de activos zombies.

## 3. Formulación Matemática Final

Sobre este subconjunto depurado ($\mathbb{S}$), se ejecuta una optimización convexa estricta diseñada para minimizar la superficie de exposición al riesgo.

### A. Función Objetivo
Minimización pura de la Varianza de la Cartera (Global Minimum Variance):

$$
\underset{w}{\text{minimizar}} \quad \sigma_p^2 = w^T \Sigma_{\mathbb{S}} w
$$

### B. Restricciones del Modelo
Se endurecen las restricciones para forzar la robustez estructural ("Antifragilidad") que faltó en la sesión del jueves:

1. **Ancla de Liquidez (90%):**
   $$
   \sum_{i \in \mathbb{S}} w_i = 0.90
   $$
   > *Justificación:* Se obliga al modelo a mantener un **10% en Cash**. Esto reduce linealmente la volatilidad global de la cartera y actúa como amortiguador ante correlaciones inesperadas.

2. **Límite de Diversificación (Hard Cap 30%):**
   $$
   0 \le w_i \le 0.30 \quad \forall i
   $$
   > *Justificación:* Tras el colapso provocado por la concentración excesiva (35% + 35%) del día anterior, esta restricción obliga al algoritmo a repartir el riesgo entre **mínimo 4 activos**, mitigando el impacto de un fallo individual.

## 4. Estructura de la Cartera Resultante

El algoritmo ha generado una estructura defensiva asimétrica ("1 Delantero + 4 Defensas"), diseñada para resistir turbulencias sin renunciar al retorno:

| Activo Seleccionado | Peso Asignado | Rol Táctico en el "Búnker" |
| :--- | :--- | :--- |
| **Activo 5 y 23** | 🧱 **Núcleo Defensivo** (~47%) | Activos de baja beta que sustituyen al Activo 36 como base estructural. |
| **Activo 6** | ⚠️ **Motor Controlado** (~20%) | Único activo de *Momentum* retenido por su descorrelación, pero con **peso reducido a la mitad** respecto al jueves para limitar la toxicidad. |
| **Activo 8 y 46** | 🛡️ **Escudos Laterales** (~22%) | Diversificadores puros que reducen la varianza global de la cartera. |
| **Cash** | ⚓ **Amortiguador** (10%) | Activo libre de riesgo para estabilizar la curva de patrimonio. |

In [2]:
import pandas as pd
import numpy as np
import cvxpy as cp

def optimizar_viernes_bunker_calidad():
    print("--- ESTRATEGIA VIERNES: BÚNKER DE CALIDAD (MIN VAR + MOMENTUM POSITIVO) ---")
    
    try:
        df = pd.read_csv('prod_long_sharpe_u50_20260116_v5_train_dataset.csv')
    except:
        return

    returns = df.values
    assets = df.columns
    n_assets = len(assets)
    
    mu = np.mean(returns, axis=0)
    Sigma = np.cov(returns, rowvar=False)
    
    # --- FILTRO 1: PURGA DE TÓXICOS ---
    # Eliminamos explícitamente los activos 6 and 36 si son volátiles
    # Y eliminamos cualquier activo con rentabilidad negativa o cero
    
    indices_candidatos = []
    print("Filtro de Seguridad:")
    for i in range(n_assets):
        # Condición: Rentabilidad positiva Y Volatilidad controlada
        vol_anual = np.sqrt(Sigma[i,i]) * np.sqrt(252)
        ret_anual = mu[i] * 252
        
        # Descartamos los que tengan volatilidad extrema (>40%) o retorno negativo
        if mu[i] > 0 and vol_anual < 0.40:
             indices_candidatos.append(i)
        else:
            # print(f"Descartado Activo {i} (Vol: {vol_anual:.1%} Ret: {ret_anual:.1%})")
            pass
            
    print(f"Activos supervivientes al filtro: {len(indices_candidatos)}")
    
    # Preparamos datos del subconjunto seguro
    indices_candidatos = np.array(indices_candidatos)
    returns_sub = returns[:, indices_candidatos]
    Sigma_sub = np.cov(returns_sub, rowvar=False)
    mu_sub = np.mean(returns_sub, axis=0)
    
    # --- OPTIMIZACIÓN: MÍNIMA VARIANZA (NO MAX SHARPE) ---
    # El objetivo es minimizar el riesgo puro. El Sharpe subirá porque el denominador bajará mucho.
    
    w = cp.Variable(len(indices_candidatos))
    risk = cp.quad_form(w, Sigma_sub)
    
    # Restricciones:
    # 1. Invertir 90% (10% Cash OBLIGATORIO)
    # 2. Max 30% por activo (Diversificación forzada)
    # 3. Max 5 Activos (Regla del concurso) - Lo forzaremos post-proceso si salen más
    
    # OJO: Aquí MINIMIZAMOS EL RIESGO, no maximizamos utilidad.
    prob = cp.Problem(cp.Minimize(risk), 
                      [cp.sum(w) == 0.90, 
                       w >= 0, 
                       w <= 0.30])
    
    prob.solve(solver=cp.SCS)
    
    if w.value is None:
        print("Error en optimización")
        return

    # --- LIMPIEZA Y SELECCIÓN FINAL (TOP 5) ---
    best_w = w.value
    top_indices_local = np.argsort(best_w)[::-1]
    
    pesos_finales = np.zeros(n_assets)
    activos_usados = 0
    
    print("\n--- CARTERA FINAL BÚNKER ---")
    for i in top_indices_local:
        peso = best_w[i]
        idx_global = indices_candidatos[i]
        
        if peso > 0.01:
            if activos_usados < 5:
                pesos_finales[idx_global] = peso
                activos_usados += 1
                
                # Métricas del activo elegido
                vol_act = np.sqrt(Sigma[idx_global, idx_global]) * np.sqrt(252)
                print(f"Activo {assets[idx_global]} (Idx {idx_global}): {peso:.2%} | Vol: {vol_act:.1%}")
            else:
                pass # Descartamos el 6º, 7º...
                
    # Renormalizar al 90% exacto los 5 elegidos
    pesos_finales = pesos_finales / np.sum(pesos_finales) * 0.90
    
    cadena = " ".join(["0.0" if p < 0.001 else str(round(p, 3)) for p in pesos_finales])
    
    print(f"\nTotal Invertido: {np.sum(pesos_finales):.1%}")
    print(f"Cash: {(1-np.sum(pesos_finales)):.1%}")
    print("\n>>> CADENA FINAL VIERNES <<<")
    print(cadena)

if __name__ == "__main__":
    optimizar_viernes_bunker_calidad()

--- ESTRATEGIA VIERNES: BÚNKER DE CALIDAD (MIN VAR + MOMENTUM POSITIVO) ---
Filtro de Seguridad:
Activos supervivientes al filtro: 33

--- CARTERA FINAL BÚNKER ---
Activo asset5 (Idx 4): 24.50% | Vol: 13.2%
Activo asset23 (Idx 22): 19.15% | Vol: 13.8%
Activo asset6 (Idx 5): 18.25% | Vol: 10.4%
Activo asset8 (Idx 7): 10.43% | Vol: 11.7%
Activo asset46 (Idx 45): 10.07% | Vol: 12.9%

Total Invertido: 90.0%
Cash: 10.0%

>>> CADENA FINAL VIERNES <<<
0.0 0.0 0.0 0.0 0.268 0.199 0.0 0.114 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.209 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.11 0.0 0.0 0.0 0.0
